In [13]:
import zipfile
import os

# 압축 해제 경로 설정
extract_path = '/home/mac/Downloads/extracted'  # 원하는 압축 해제 경로로 변경하세요

# 압축 해제 경로가 없으면 생성
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# CP949(한국어 인코딩) 사용하여 압축 해제
with zipfile.ZipFile('/home/mac/Downloads/20250328102438.zip', 'r') as zip_ref:
    # 파일 목록 확인
    file_list = zip_ref.namelist()
    
    # 각 파일 압축 해제
    for file in file_list:
        # CP949로 디코딩하여 원래 파일명 복원
        filename = file.encode('cp437').decode('cp949')
        
        # 압축 해제
        with zip_ref.open(file) as source:
            with open(os.path.join(extract_path, filename), 'wb') as target:
                target.write(source.read())

print(f"압축 해제 완료. 파일은 {extract_path}에 저장되었습니다.")

압축 해제 완료. 파일은 /home/mac/Downloads/extracted에 저장되었습니다.


In [4]:
import pandas as pd

card_df = pd.read_csv("/home/mac/project_ws/eda-repo-5/visualization/weather_years/가맹점_카드매출액_2020년_1월.csv")

In [ ]:
card_df = card_df.iloc[:, :-1]

,Date,전체
0,2020-01-10,-6.4
1,2020-01-17,-0.8
2,2020-01-24,6.6
3,2020-01-31,0.6
4,2020-02-07,-19.3
...,...,...
266,2025-02-14,28.4
267,2025-02-21,17.5
268,2025-02-28,60.5
269,2025-03-07,5.6


In [ ]:
card_df

In [86]:
import zipfile
import os
import pandas as pd
import re

# 압축 파일 경로
zip_path = '/home/mac/Downloads/20250328102438.zip'

# 압축 해제 경로 설정
extract_path = '/home/mac/Downloads/extracted'

# 압축 해제 경로가 없으면 생성
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# 데이터프레임 저장 딕셔너리
dataframes = {}

# 한글 파일명을 영어 파일명으로 변환하는 함수
def convert_to_english_name(filename):
    # 확장자 분리
    base_name, ext = os.path.splitext(filename)
    
    # 한글 또는 특수문자를 제거하고 영어와 숫자만 유지
    english_name = re.sub(r'[^a-zA-Z0-9]', '_', base_name)
    
    # 언더스코어가 연속으로 있는 경우 하나로 변경
    english_name = re.sub(r'_+', '_', english_name)
    
    # 앞뒤 언더스코어 제거
    english_name = english_name.strip('_')
    
    # 비어있는 경우 기본값 설정
    if not english_name:
        english_name = "file"
    
    return english_name

# 압축 파일 처리
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    # 파일 목록 확인
    file_list = zip_ref.namelist()
    print("압축 파일 내 파일 목록:")
    for file in file_list:
        print(f"  - {file}")
    
    # 각 파일 처리
    for file in file_list:
        # 디렉토리는 건너뛰기
        if file.endswith('/'):
            continue
        
        try:
            # CP949로 디코딩하여 원래 파일명 복원 시도
            try:
                decoded_filename = file.encode('cp437').decode('cp949', errors='replace')
            except:
                decoded_filename = file
            
            # 영어 변수명 생성
            english_name = convert_to_english_name(decoded_filename)
            var_name = f"{english_name}_df"
            
            # 파일 확장자 확인
            _, ext = os.path.splitext(file)
            
            # 임시 파일 경로
            temp_path = os.path.join(extract_path, os.path.basename(file))
            
            # 파일 추출
            with zip_ref.open(file) as source:
                with open(temp_path, 'wb') as target:
                    target.write(source.read())
            
            # 파일 유형에 따라 데이터프레임 생성
            if ext.lower() in ['.xlsx', '.xls']:
                # 엑셀 파일
                df = pd.read_excel(temp_path, engine='openpyxl' if ext.lower() == '.xlsx' else 'xlrd')
                dataframes[var_name] = df
                # 전역 변수로 등록
                globals()[var_name] = df
                print(f"\n엑셀 파일 '{decoded_filename}' → '{var_name}' 변환 완료")
                print(f"  - 크기: {df.shape[0]} 행 x {df.shape[1]} 열")
                print(f"  - 처음 5행:\n{df.head()}")
                
            elif ext.lower() == '.csv':
                # CSV 파일 (인코딩 자동 감지)
                for encoding in ['utf-8', 'cp949', 'euc-kr']:
                    try:
                        df = pd.read_csv(temp_path, encoding=encoding)
                        dataframes[var_name] = df
                        # 전역 변수로 등록
                        globals()[var_name] = df
                        print(f"\nCSV 파일 '{decoded_filename}' → '{var_name}' 변환 완료 (인코딩: {encoding})")
                        print(f"  - 크기: {df.shape[0]} 행 x {df.shape[1]} 열")
                        print(f"  - 처음 5행:\n{df.head()}")
                        break
                    except UnicodeDecodeError:
                        continue
                    except Exception as e:
                        print(f"  - CSV 처리 오류 ({encoding}): {str(e)}")
            
            # 임시 파일 삭제
            os.remove(temp_path)
            
        except Exception as e:
            print(f"\n파일 '{file}' 처리 중 오류: {str(e)}")

print(f"\n생성된 데이터프레임 목록:")
for name in dataframes.keys():
    print(f"  - {name}: {globals()[name].shape[0]} 행 x {globals()[name].shape[1]} 열")

압축 파일 내 파일 목록:
  - ├╗┐ε╚┐└┌╡┐_░¡╝÷╟ⁿ┼┬_202401_202412.csv
  - ├╗┐ε╚┐└┌╡┐_╜└╡╡_202401_202412.csv
  - ├╗┐ε╚┐└┌╡┐_░¡╝÷_202401_202412.csv
  - ├╗┐ε╚┐└┌╡┐_╟╧┤├╗≤┼┬_202401_202412.csv
  - ├╗┐ε╚┐└┌╡┐_▒Γ┐┬_202401_202412.csv
  - ├╗┐ε╚┐└┌╡┐_╟│╝╙_202401_202412.csv

CSV 파일 '청운효자동_강수형태_202401_202412.csv' → '202401_202412_df' 변환 완료 (인코딩: utf-8)
  - 크기: 8795 행 x 3 열
  - 처음 5행:
   format: day   hour  value location:60_127 Start : 20240101 
0            1    0.0                                      0.0
1            1  100.0                                      0.0
2            1  200.0                                      0.0
3            1  300.0                                      0.0
4            1  400.0                                      0.0

CSV 파일 '청운효자동_습도_202401_202412.csv' → '202401_202412_df' 변환 완료 (인코딩: utf-8)
  - 크기: 8795 행 x 3 열
  - 처음 5행:
   format: day   hour  value location:60_127 Start : 20240101 
0            1    0.0                                     94.0
1            1  100.0    

In [87]:
dataframes

{'202401_202412_df':       format: day    hour  value location:60_127 Start : 20240101 
 0               1     0.0                                      1.6
 1               1   100.0                                      2.0
 2               1   200.0                                      1.9
 3               1   300.0                                      1.3
 4               1   400.0                                      1.4
 ...           ...     ...                                      ...
 8790           31  1900.0                                      2.2
 8791           31  2000.0                                      2.4
 8792           31  2100.0                                      2.2
 8793           31  2200.0                                      1.8
 8794           31  2300.0                                      1.7
 
 [8795 rows x 3 columns]}

In [88]:
import pandas as pd

df_vars = [var_name for var_name, var_value in globals().items() 
           if isinstance(var_value, pd.DataFrame)]

print(f"데이터프레임 변수 목록 ({len(df_vars)}개):")
for var_name in df_vars:
    print(f"- {var_name}")

데이터프레임 변수 목록 (38개):
- _1
- card_df
- _5
- _7
- _10
- df
- spot_df
- cafe_df
- restaurant_df
- play_df
- region_counts
- region_counts_sorted
- 202401_202412_df
- _21
- result
- _23
- _26
- _34
- _42
- before_start
- after_start
- before_feb
- after_feb
- segment_df
- second_segment
- _47
- _49
- _54
- _58
- _64
- _66
- _70
- _72
- _74
- _81
- df_extended
- _84
- _85


In [89]:
df.columns

Index([' format: day', 'hour', 'value location:60_127 Start : 20240101 '], dtype='object')

In [90]:
df = df.rename(columns={' format: day': '일', 'value location:60_127 Start : 20240101 ':'강수량'})

In [91]:
df = df.drop('hour', axis=1)

In [92]:
df

,일,강수량
0,1,1.6
1,1,2.0
2,1,1.9
3,1,1.3
4,1,1.4
...,...,...
8790,31,2.2
8791,31,2.4
8792,31,2.2
8793,31,1.8


In [93]:
import pandas as pd
import numpy as np

def group_data_by_7(df):
    """
    데이터프레임의 행을 7개씩 그룹화하고 각 그룹의 일(최대값), 강수량(평균)을 계산
    
    Parameters:
    df : 강수량 데이터가 포함된 데이터프레임
    
    Returns:
    DataFrame: 7개씩 그룹화된 결과
    """
    # 데이터프레임 복사
    df_copy = df.copy()
    
    # 결과를 저장할 리스트
    result_list = []
    
    # 인덱스를 7개씩 그룹화
    df_copy['group_idx'] = df_copy.index // 7
    
    # 그룹별로 데이터 처리
    grouped = df_copy.groupby('group_idx')
    
    for group_idx, group in grouped:
        # ID 범위
        start_id = group.index.min()
        end_id = group.index.max()
        id_range = f"{start_id}~{end_id}"
        
        # 그룹 내 최대 일값 계산
        # 문자열과 숫자가 섞여 있을 수 있으므로 안전하게 처리
        try:
            # 일(day) 열을 숫자로 변환 시도
            numeric_days = pd.to_numeric(group['일'], errors='coerce')
            # NaN 제외하고 최대값 구하기
            if not numeric_days.dropna().empty:
                max_day = numeric_days.dropna().max()
            else:
                max_day = group['일'].iloc[0]  # 변환할 수 없는 경우 첫 번째 값 사용
        except:
            # 오류 발생 시 첫 번째 값 사용
            max_day = group['일'].iloc[0]
        
        # 강수량 평균 계산
        avg_rainfall = group['강수량'].mean()
        
        # 결과에 추가
        result_list.append({
            '그룹범위': id_range,
            '일': max_day,
            '강수량': round(avg_rainfall, 2)
        })
    
    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(result_list)
    return result_df

# 이미지에 보이는 데이터로 예시 생성
data = {
    '일': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    '강수량': [1.6, 2.0, 1.9, 1.3, 1.4, 2.1, 2.5, 2.1, 1.3, 1.5]
}

# 데이터프레임 생성 (인덱스를 ID로 사용)
index_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
df = pd.DataFrame(data, index=index_values)

# 데이터 처리 및 결과 출력
result = group_data_by_7(df)
print("ID 7개씩 그룹화 결과:")
print(result)

# 실제 CSV 파일에서 데이터를 불러와 사용하는 예시:
"""
# CSV 파일에서 데이터 로드
df_from_csv = pd.read_csv('your_data.csv', index_col=0)

# 데이터 처리
result = group_data_by_7(df_from_csv)
print(result)

# 결과 저장
result.to_csv('grouped_data.csv', index=False)
"""

ID 7개씩 그룹화 결과:
  그룹범위  일   강수량
0  0~6  1  1.83
1  7~9  1  1.63


"\n# CSV 파일에서 데이터 로드\ndf_from_csv = pd.read_csv('your_data.csv', index_col=0)\n\n# 데이터 처리\nresult = group_data_by_7(df_from_csv)\nprint(result)\n\n# 결과 저장\nresult.to_csv('grouped_data.csv', index=False)\n"

In [94]:
df

,일,강수량
0,1,1.6
1,1,2.0
2,1,1.9
3,1,1.3
4,1,1.4
5,1,2.1
6,1,2.5
7,1,2.1
8,1,1.3
9,1,1.5


In [95]:
import pandas as pd
import numpy as np

def group_data_by_7(df):
    """
    데이터프레임의 행을 7개씩 그룹화하고 각 그룹의 일(최대값), 강수량(평균)을 계산
    
    Parameters:
    df : 강수량 데이터가 포함된 데이터프레임
    
    Returns:
    DataFrame: 7개씩 그룹화된 결과
    """
    # 데이터프레임 복사
    df_copy = df.copy()
    
    # 결과를 저장할 리스트
    result_list = []
    
    # 인덱스를 7개씩 그룹화
    df_copy['group_idx'] = df_copy.index // 7
    
    # 그룹별로 데이터 처리
    grouped = df_copy.groupby('group_idx')
    
    for group_idx, group in grouped:
        # ID 범위
        start_id = group.index.min()
        end_id = group.index.max()
        id_range = f"{start_id}~{end_id}"
        
        # 그룹 내 최대 일값 계산
        # 문자열과 숫자가 섞여 있을 수 있으므로 안전하게 처리
        try:
            # 일(day) 열을 숫자로 변환 시도
            numeric_days = pd.to_numeric(group['일'], errors='coerce')
            # NaN 제외하고 최대값 구하기
            if not numeric_days.dropna().empty:
                max_day = numeric_days.dropna().max()
            else:
                max_day = group['일'].iloc[0]  # 변환할 수 없는 경우 첫 번째 값 사용
        except:
            # 오류 발생 시 첫 번째 값 사용
            max_day = group['일'].iloc[0]
        
        # 강수량 평균 계산
        avg_rainfall = group['강수량'].mean()
        
        # 결과에 추가
        result_list.append({
            '그룹범위': id_range,
            '일': max_day,
            '강수량': round(avg_rainfall, 2)
        })
    
    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(result_list)
    return result_df

# 이미지에 보이는 데이터로 예시 생성
data = {
    '일': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    '강수량': [1.6, 2.0, 1.9, 1.3, 1.4, 2.1, 2.5, 2.1, 1.3, 1.5]
}

# 데이터프레임 생성 (인덱스를 ID로 사용)
index_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
df = pd.DataFrame(data, index=index_values)

# 데이터 처리 및 결과 출력
result = group_data_by_7(df)
print("ID 7개씩 그룹화 결과:")
print(result)

# 실제 CSV 파일에서 데이터를 불러와 사용하는 예시:
"""
# CSV 파일에서 데이터 로드
df_from_csv = pd.read_csv('your_data.csv', index_col=0)

# 데이터 처리
result = group_data_by_7(df_from_csv)
print(result)

# 결과 저장
result.to_csv('grouped_data.csv', index=False)
"""

ID 7개씩 그룹화 결과:
  그룹범위  일   강수량
0  0~6  1  1.83
1  7~9  1  1.63


"\n# CSV 파일에서 데이터 로드\ndf_from_csv = pd.read_csv('your_data.csv', index_col=0)\n\n# 데이터 처리\nresult = group_data_by_7(df_from_csv)\nprint(result)\n\n# 결과 저장\nresult.to_csv('grouped_data.csv', index=False)\n"

In [96]:
df

,일,강수량
0,1,1.6
1,1,2.0
2,1,1.9
3,1,1.3
4,1,1.4
5,1,2.1
6,1,2.5
7,1,2.1
8,1,1.3
9,1,1.5


In [97]:
import pandas as pd
import os

def load_weather_dataframes():
    """
    6개의 CSV 파일을 각각 별도의 데이터프레임으로 로드
    
    Returns:
    tuple: (weather1, weather2, weather3, weather4, weather5, weather6) 6개의 데이터프레임
    """
    # 기본 경로 설정
    base_path = '/home/mac/Downloads/20250328102438'
    
    # 파일 경로 목록
    file_paths = [
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_╜└╡╡_202401_202412.csv'),      # 첫 번째 파일 -> weather1
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_╟│╝╙_202401_202412.csv'),      # 두 번째 파일 -> weather2
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_╟╧┤├╗≤┼┬_202401_202412.csv'),  # 세 번째 파일 -> weather3
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_░¡╝÷_202401_202412.csv'),      # 네 번째 파일 -> weather4
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_░¡╝÷╟ⁿ┼┬_202401_202412.csv'), # 다섯 번째 파일 -> weather5
        os.path.join(base_path, '├╗┐ε╚┐└┌╡┐_▒Γ┐┬_202401_202412.csv')       # 여섯 번째 파일 -> weather6
    ]
    
    # 결과 데이터프레임을 저장할 변수 초기화
    weather_dfs = [None] * 6
    
    # 각 파일을 로드하여 데이터프레임에 할당
    for i, file_path in enumerate(file_paths):
        try:
            # 파일 존재 여부 확인
            if not os.path.exists(file_path):
                print(f"경고: '{file_path}' 파일을 찾을 수 없습니다.")
                continue
                
            # 여러 인코딩 방식 시도
            encodings = ['utf-8', 'cp949', 'euc-kr']
            for encoding in encodings:
                try:
                    weather_dfs[i] = pd.read_csv(file_path, encoding=encoding)
                    print(f"'{file_path}' 파일을 '{encoding}' 인코딩으로 로드 완료")
                    break
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    print(f"'{file_path}' 파일 로드 중 오류 발생: {e}")
                    break
            
            # 모든 인코딩으로 시도해도 실패한 경우
            if weather_dfs[i] is None:
                print(f"'{file_path}' 파일을 로드할 수 없습니다. 인코딩 문제가 있을 수 있습니다.")
                
        except Exception as e:
            print(f"'{file_path}' 파일 처리 중 예상치 못한 오류 발생: {e}")
    
    # 결과 확인
    weather1, weather2, weather3, weather4, weather5, weather6 = weather_dfs
    
    print("\n데이터프레임 정보:")
    print(f"weather1: {weather1.shape if weather1 is not None else 'None'}")
    print(f"weather2: {weather2.shape if weather2 is not None else 'None'}")
    print(f"weather3: {weather3.shape if weather3 is not None else 'None'}")
    print(f"weather4: {weather4.shape if weather4 is not None else 'None'}")
    print(f"weather5: {weather5.shape if weather5 is not None else 'None'}")
    print(f"weather6: {weather6.shape if weather6 is not None else 'None'}")
    
    return weather1, weather2, weather3, weather4, weather5, weather6

# 함수 사용 예시
"""
# 데이터프레임 로드
weather1, weather2, weather3, weather4, weather5, weather6 = load_weather_dataframes()

# 각 데이터프레임 확인
for i, df in enumerate([weather1, weather2, weather3, weather4, weather5, weather6], 1):
    if df is not None:
        print(f"\nweather{i} 정보:")
        print(f"행 수: {df.shape[0]}, 열 수: {df.shape[1]}")
        print(f"컬럼 목록: {df.columns.tolist()}")
        print(f"첫 3행:\n{df.head(3)}")
"""

'\n# 데이터프레임 로드\nweather1, weather2, weather3, weather4, weather5, weather6 = load_weather_dataframes()\n\n# 각 데이터프레임 확인\nfor i, df in enumerate([weather1, weather2, weather3, weather4, weather5, weather6], 1):\n    if df is not None:\n        print(f"\nweather{i} 정보:")\n        print(f"행 수: {df.shape[0]}, 열 수: {df.shape[1]}")\n        print(f"컬럼 목록: {df.columns.tolist()}")\n        print(f"첫 3행:\n{df.head(3)}")\n'

In [99]:
# 데이터프레임 로드
weather1, weather2, weather3, weather4, weather5, weather6 = load_weather_dataframes()

# 각 데이터프레임 확인
for i, df in enumerate([weather1, weather2, weather3, weather4, weather5, weather6], 1):
    if df is not None:
        print(f"weather{i} 정보:")
        print(f"행 수: {df.shape[0]}, 열 수: {df.shape[1]}")
        print(f"첫 3행:\n{df.head(3)}")
        print("\n")

'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_╜└╡╡_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료
'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_╟│╝╙_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료
'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_╟╧┤├╗≤┼┬_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료
'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_░¡╝÷_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료
'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_░¡╝÷╟ⁿ┼┬_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료
'/home/mac/Downloads/20250328102438/├╗┐ε╚┐└┌╡┐_▒Γ┐┬_202401_202412.csv' 파일을 'utf-8' 인코딩으로 로드 완료

데이터프레임 정보:
weather1: (8795, 3)
weather2: (8795, 3)
weather3: (0, 3)
weather4: (8795, 3)
weather5: (8795, 3)
weather6: (8795, 3)
weather1 정보:
행 수: 8795, 열 수: 3
첫 3행:
   format: day   hour  value location:60_127 Start : 20240101 
0            1    0.0                                     94.0
1            1  100.0                                     80.0
2            1  200.0                                     72.0


we

In [100]:
weather1

,format: day,hour,value location:60_127 Start : 20240101
0,1,0.0,94.0
1,1,100.0,80.0
2,1,200.0,72.0
3,1,300.0,68.0
4,1,400.0,69.0
...,...,...,...
8790,31,1900.0,69.0
8791,31,2000.0,68.0
8792,31,2100.0,68.0
8793,31,2200.0,67.0


In [101]:
weather2

,format: day,hour,value location:60_127 Start : 20240101
0,1,0.0,1.6
1,1,100.0,2.0
2,1,200.0,1.9
3,1,300.0,1.3
4,1,400.0,1.4
...,...,...,...
8790,31,1900.0,2.2
8791,31,2000.0,2.4
8792,31,2100.0,2.2
8793,31,2200.0,1.8
